In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd

# Extract

In [9]:
#my sample google sheet
sheet_id = '1xo1_DZEVBiVz39S5Ln3CSnagbPE78cvBDKtwqtC3hKI'
sheet_names = ['Contacts','Business','Cases','Bank_Account','Withdraw_Info']

#extract all tables from the sheet
contact_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Contacts"
df_contact = pd.read_csv(contact_url)

business_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Business"
df_business = pd.read_csv(business_url)

case_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Cases"
df_case = pd.read_csv(case_url)

bank_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Bank_Account"
df_bank = pd.read_csv(bank_url)

withdraw_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Withdraw_Info"
df_withdraw = pd.read_csv(withdraw_url)

In [10]:
df_withdraw

,withdraw_id,contact_id,bank_account_id,withdraw_amount,withdraw_date,withdraw_time,withdraw_type,withdraw_location
0,W001,C001,BANK001,NaN,NaN,NaN,NaN,NaN
1,W002,C002,BANK002,NaN,NaN,NaN,NaN,NaN
2,W003,C003,BANK003,NaN,NaN,NaN,NaN,NaN
3,W004,C004,BANK004,NaN,NaN,NaN,NaN,NaN
4,W005,C005,BANK005,NaN,NaN,NaN,NaN,NaN
5,W006,C006,BANK006,NaN,NaN,NaN,NaN,NaN
6,W007,C007,BANK007,NaN,NaN,NaN,NaN,NaN
7,W008,C008,BANK008,NaN,NaN,NaN,NaN,NaN
8,W009,C009,BANK009,NaN,NaN,NaN,NaN,NaN
9,W010,C010,BANK010,NaN,NaN,NaN,NaN,NaN
